In [1]:
from pathlib import Path
import json

from rich.progress import track
from collections import defaultdict

from openff.toolkit.utils.exceptions import UndefinedStereochemistryError
from openff.evaluator.datasets.thermoml import ThermoMLDataSet

In [2]:
PROP_BROWSE_URL = 'https://trc.nist.gov/ThermoML/Browse'
osm_coeff_dois = [
"10.1016/j.jct.2013.08.018",
"10.1016/j.fluid.2016.02.030",
"10.1016/j.tca.2012.09.009",
"10.1016/j.fluid.2012.05.002",
"10.1016/j.fluid.2006.09.025",
"10.1016/j.jct.2013.05.027",
"10.1016/j.jct.2009.06.024",
"10.1016/j.jct.2011.03.012",
"10.1016/j.jct.2008.12.021",
"10.1021/je400821q",
"10.1016/j.fluid.2005.12.016",
"10.1016/j.fluid.2011.09.037",
"10.1016/j.jct.2013.01.032",
"10.1016/j.jct.2016.01.026",
"10.1021/je4008475",
"10.1016/j.jct.2013.09.041",
"10.1016/j.jct.2011.01.010",
"10.1021/je2009413",
"10.1016/j.jct.2013.12.002",
"10.1016/j.jct.2009.12.001",
"10.1021/je900216m",
"10.1016/j.jct.2012.02.007",
"10.1016/j.jct.2011.06.018",
"10.1016/j.jct.2008.05.011",
"10.1016/j.fluid.2014.02.004",
"10.1021/je500772z",
"10.1021/je100720x",
"10.1016/j.fluid.2007.03.019",
"10.1016/j.fluid.2015.07.004",
"10.1016/j.jct.2008.08.011",
"10.1016/j.jct.2012.01.007",
"10.1016/j.fluid.2006.08.020",
"10.1016/j.fluid.2015.08.035",
"10.1016/j.jct.2008.07.011",
"10.1016/j.fluid.2009.02.007",
"10.1021/je3006985",
"10.1016/j.fluid.2006.10.012",
"10.1016/j.jct.2018.08.021",
"10.1016/j.jct.2017.05.006",
"10.1016/j.jct.2014.11.014",
"10.1016/j.jct.2016.07.003",
"10.1021/acs.jced.5b00184",
"10.1021/je800609u",
"10.1021/acs.jced.8b00400",
"10.1016/j.jct.2014.10.008",
"10.1016/j.fluid.2012.11.011",
"10.1021/je100864e",
"10.1016/j.fluid.2010.03.012",
"10.1021/je050348f",
"10.1016/j.jct.2016.06.034",
"10.1007/s10765-009-0566-6",
"10.1016/j.fluid.2004.11.022",
"10.1021/je800307g",
"10.1021/je100760v",
"10.1021/je700751a",
"10.1016/j.jct.2019.105878",
"10.1016/j.fluid.2015.03.014",
"10.1016/j.jct.2013.03.003",
"10.1016/j.jct.2017.09.012",
"10.1021/je900457z",
"10.1021/acs.jced.9b00112",
"10.1016/j.jct.2014.07.010",
"10.1021/je500271z",
"10.1016/j.jct.2013.08.018",
"10.1016/j.fluid.2016.02.030",
"10.1016/j.tca.2012.09.009",
"10.1016/j.fluid.2012.05.002",
"10.1016/j.fluid.2006.09.025",
"10.1016/j.jct.2013.05.027",
"10.1016/j.jct.2009.06.024",
"10.1016/j.jct.2011.03.012",
"10.1016/j.jct.2008.12.021",
"10.1021/je400821q",
"10.1016/j.fluid.2005.12.016",
"10.1016/j.fluid.2011.09.037",
"10.1016/j.jct.2013.01.032",
"10.1016/j.jct.2016.01.026",
"10.1021/je4008475",
"10.1016/j.jct.2013.09.041",
"10.1016/j.jct.2011.01.010",
"10.1021/je2009413",
"10.1016/j.jct.2013.12.002",
"10.1016/j.jct.2009.12.001",
"10.1021/je900216m",
"10.1016/j.jct.2012.02.007",
"10.1016/j.jct.2011.06.018",
"10.1016/j.jct.2008.05.011",
"10.1016/j.fluid.2014.02.004",
"10.1021/je500772z",
"10.1021/je100720x",
"10.1016/j.fluid.2007.03.019",
"10.1016/j.fluid.2015.07.004",
"10.1016/j.jct.2008.08.011",
"10.1016/j.jct.2012.01.007",
"10.1016/j.fluid.2006.08.020",
"10.1016/j.fluid.2015.08.035",
"10.1016/j.jct.2008.07.011",
"10.1016/j.fluid.2009.02.007",
"10.1021/je3006985",
"10.1016/j.fluid.2006.10.012",
"10.1016/j.jct.2018.08.021",
"10.1016/j.jct.2017.05.006",
"10.1016/j.jct.2014.11.014",
"10.1016/j.jct.2016.07.003",
"10.1021/acs.jced.5b00184",
"10.1021/je800609u",
"10.1021/acs.jced.8b00400",
"10.1016/j.jct.2014.10.008",
"10.1016/j.fluid.2012.11.011",
"10.1021/je100864e",
"10.1016/j.fluid.2010.03.012",
"10.1021/je050348f",
"10.1016/j.jct.2016.06.034",
"10.1007/s10765-009-0566-6",
"10.1016/j.fluid.2004.11.022",
"10.1021/je800307g",
"10.1021/je100760v",
"10.1021/je700751a",
"10.1016/j.jct.2019.105878",
"10.1016/j.fluid.2015.03.014",
"10.1016/j.jct.2013.03.003",
"10.1016/j.jct.2017.09.012",
"10.1021/je900457z",
"10.1021/acs.jced.9b00112",
"10.1016/j.jct.2014.07.010",
"10.1021/je500271z",
"10.1016/j.jct.2013.08.018",
"10.1016/j.fluid.2016.02.030",
"10.1016/j.tca.2012.09.009",
"10.1016/j.fluid.2012.05.002",
"10.1016/j.fluid.2006.09.025",
"10.1016/j.jct.2013.05.027",
"10.1016/j.jct.2009.06.024",
"10.1016/j.jct.2011.03.012",
"10.1016/j.jct.2008.12.021",
"10.1021/je400821q",
"10.1016/j.fluid.2005.12.016",
"10.1016/j.fluid.2011.09.037",
"10.1016/j.jct.2013.01.032",
"10.1016/j.jct.2016.01.026",
"10.1021/je4008475",
"10.1016/j.jct.2013.09.041",
"10.1016/j.jct.2011.01.010",
"10.1021/je2009413",
"10.1016/j.jct.2013.12.002",
"10.1016/j.jct.2009.12.001",
"10.1021/je900216m",
"10.1016/j.jct.2012.02.007",
"10.1016/j.jct.2011.06.018",
"10.1016/j.jct.2008.05.011",
"10.1016/j.fluid.2014.02.004",
"10.1021/je500772z",
"10.1021/je100720x",
"10.1016/j.fluid.2007.03.019",
"10.1016/j.fluid.2015.07.004",
"10.1016/j.jct.2008.08.011",
"10.1016/j.jct.2012.01.007",
"10.1016/j.fluid.2006.08.020",
"10.1016/j.fluid.2015.08.035",
"10.1016/j.jct.2008.07.011",
"10.1016/j.fluid.2009.02.007",
"10.1021/je3006985",
"10.1016/j.fluid.2006.10.012",
"10.1016/j.jct.2018.08.021",
"10.1016/j.jct.2017.05.006",
"10.1016/j.jct.2014.11.014",
"10.1016/j.jct.2016.07.003",
"10.1021/acs.jced.5b00184",
"10.1021/je800609u",
"10.1021/acs.jced.8b00400",
"10.1016/j.jct.2014.10.008",
"10.1016/j.fluid.2012.11.011",
"10.1021/je100864e",
"10.1016/j.fluid.2010.03.012",
"10.1021/je050348f",
"10.1016/j.jct.2016.06.034",
"10.1007/s10765-009-0566-6",
"10.1016/j.fluid.2004.11.022",
"10.1021/je800307g",
"10.1021/je100760v",
"10.1021/je700751a",
"10.1016/j.jct.2019.105878",
"10.1016/j.fluid.2015.03.014",
"10.1016/j.jct.2013.03.003",
"10.1016/j.jct.2017.09.012",
"10.1021/je900457z",
"10.1021/acs.jced.9b00112",
"10.1016/j.jct.2014.07.010",
"10.1021/je500271z",
"10.1016/j.jct.2013.08.018",
"10.1016/j.fluid.2016.02.030",
"10.1016/j.tca.2012.09.009",
"10.1016/j.fluid.2012.05.002",
"10.1016/j.fluid.2006.09.025",
"10.1016/j.jct.2013.05.027",
"10.1016/j.jct.2009.06.024",
"10.1016/j.jct.2011.03.012",
"10.1016/j.jct.2008.12.021",
"10.1021/je400821q",
"10.1016/j.fluid.2005.12.016",
"10.1016/j.fluid.2011.09.037",
"10.1016/j.jct.2013.01.032",
"10.1016/j.jct.2016.01.026",
"10.1021/je4008475",
"10.1016/j.jct.2013.09.041",
"10.1016/j.jct.2011.01.010",
"10.1021/je2009413",
"10.1016/j.jct.2013.12.002",
"10.1016/j.jct.2009.12.001",
"10.1021/je900216m",
"10.1016/j.jct.2012.02.007",
"10.1016/j.jct.2011.06.018",
"10.1016/j.jct.2008.05.011",
"10.1016/j.fluid.2014.02.004",
"10.1021/je500772z",
"10.1021/je100720x",
"10.1016/j.fluid.2007.03.019",
"10.1016/j.fluid.2015.07.004",
"10.1016/j.jct.2008.08.011",
"10.1016/j.jct.2012.01.007",
"10.1016/j.fluid.2006.08.020",
"10.1016/j.fluid.2015.08.035",
"10.1016/j.jct.2008.07.011",
"10.1016/j.fluid.2009.02.007",
"10.1021/je3006985",
"10.1016/j.fluid.2006.10.012",
"10.1016/j.jct.2018.08.021",
"10.1016/j.jct.2017.05.006",
"10.1016/j.jct.2014.11.014",
"10.1016/j.jct.2016.07.003",
"10.1021/acs.jced.5b00184",
"10.1021/je800609u",
"10.1021/acs.jced.8b00400",
"10.1016/j.jct.2014.10.008",
"10.1016/j.fluid.2012.11.011",
"10.1021/je100864e",
"10.1016/j.fluid.2010.03.012",
"10.1021/je050348f",
"10.1016/j.jct.2016.06.034",
"10.1007/s10765-009-0566-6",
"10.1016/j.fluid.2004.11.022",
"10.1021/je800307g",
"10.1021/je100760v",
"10.1021/je700751a",
"10.1016/j.jct.2019.105878",
"10.1016/j.fluid.2015.03.014",
"10.1016/j.jct.2013.03.003",
"10.1016/j.jct.2017.09.012",
"10.1021/je900457z",
"10.1021/acs.jced.9b00112",
"10.1016/j.jct.2014.07.010",
"10.1021/je500271z",
]

with Path('osm_coeff_dois.json').open('w') as file:
    json.dump(osm_coeff_dois, file, indent=4)

In [3]:
sorted_dois = defaultdict(list)
for doi in track(osm_coeff_dois, description='Filtering DOIs...'):
    try:
        dataset = ThermoMLDataSet.from_doi(doi)
        sorted_dois['working'].append(doi)
    except UndefinedStereochemistryError:
        sorted_dois['stereo_fail'].append(doi)
    except Exception as other_exc:
        sorted_dois[other_exc.__class__.__name__].append(doi)

with Path('sorted_dois.json').open('w') as file:
    json.dump(sorted_dois, file, indent=4)

Output()

[14:35:25] Explicit valence for atom # 6 Si, 8, is greater than permitted

[14:35:25] ERROR: Explicit valence for atom # 6 Si, 8, is greater than permitted

[14:35:36] Explicit valence for atom # 6 Si, 8, is greater than permitted

[14:35:36] ERROR: Explicit valence for atom # 6 Si, 8, is greater than permitted

[14:35:48] Explicit valence for atom # 6 Si, 8, is greater than permitted

[14:35:48] ERROR: Explicit valence for atom # 6 Si, 8, is greater than permitted